In [3]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import cv2
from torchvision import models
from torchsummary import summary

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset
uframes = []
vidcap = cv2.VideoCapture('spacecraft_final.avi')
success,image = vidcap.read()
while success:
    uframes.append(image)   
    success,image = vidcap.read()

# cap = cv2.VideoCapture('video_drone_tank.avi')


# ret, frame = cap.read()

# uframes.append(frame)
# flag = 0
# z = 0
# while(z<30):
#     ret, frame = cap.read()
#     #np_frame = cv2.imread('video', frame) # does not work
#     #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
#     #print(np_frame.shape)
#     for a in uframes:
#         if (np.array(a).any() != None):
#             print("a: ",np.shape(a))
#             print("frame: ",np.shape(a))
#             difference = cv2.subtract(np.array(a), np.array(frame))    
#             result = not np.any(difference)
#             #print("inside for")
#             if result is True:
#                 flag = 1
#                 #print("setting flag")
#                 break
#     if(not flag):
#         #print("appending")
#         uframes.append(frame)
#     z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))

print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))



#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (100,100), color = '#9b7653')
# img.paste(im1,(5,75),im1)
# img.paste(im1,(10,55),im1)
# img.paste(im1,(25,50),im1)
# img.paste(im2,(75,5),im2)
# img.paste(im2,(60,30),im2)
# img.paste(im2,(90,20),im2)
# img.paste(im3,(75,60),im3)
# img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)

#y0 = np.array(np.transpose(np.asarray(y0),(0,3,1,2)),dtype=np.float32)
#y1 = np.array(np.transpose(np.asarray(y1),(0,3,1,2)),dtype=np.float32)
print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=5)

#         # Convolution 2
#         self.cnn2 = nn.Conv2d(in_channels=16, out_channels=3, kernel_size=5, stride=1, padding=2)
#         self.relu2 = nn.ReLU()

#         # Max pool 2
#         self.maxpool2 = nn.MaxPool2d(kernel_size=5)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(11552, 15)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        #out = self.cnn2(out)
        #out = self.relu2(out)

        # Max pool 2 
        #out = self.maxpool2(out)
        im_out = out
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        out = self.sigmoid1(out)
        return im_out,out
    
model = CNNModel()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x)[1],model(y)[1])
        # ===================backward====================
 
        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    #print("shape: ", np.shape(model(x[0].reshape(1,3,100, 100))[0]))
#     print("x: ", model(x[0].reshape(1,3,100, 100)))
#     print("y:", model(y[0].reshape(1,3,100, 100)))
#     print("b:", model(b.reshape(1,3,100, 100)))


(100, 100, 3)
length:  264
(20000, 3, 100, 100)
epoch [1/200], loss:-1.2410
epoch [2/200], loss:-1.3223
epoch [3/200], loss:-1.3238
epoch [4/200], loss:-1.3299
epoch [5/200], loss:-1.4103
epoch [6/200], loss:-1.1987
epoch [7/200], loss:-1.3091
epoch [8/200], loss:-1.3856
epoch [9/200], loss:-1.2810
epoch [10/200], loss:-1.3361
epoch [11/200], loss:-1.4756
epoch [12/200], loss:-1.3370
epoch [13/200], loss:-1.4192
epoch [14/200], loss:-1.3792
epoch [15/200], loss:-1.5616
epoch [16/200], loss:-1.1567
epoch [17/200], loss:-1.2076
epoch [18/200], loss:-1.3346
epoch [19/200], loss:-1.4412
epoch [20/200], loss:-1.5291
epoch [21/200], loss:-1.4660
epoch [22/200], loss:-1.3675
epoch [23/200], loss:-1.4957
epoch [24/200], loss:-1.4785
epoch [25/200], loss:-1.2696
epoch [26/200], loss:-1.4311
epoch [27/200], loss:-1.2647
epoch [28/200], loss:-1.3910
epoch [29/200], loss:-1.3640
epoch [30/200], loss:-1.4238
epoch [31/200], loss:-1.3351
epoch [32/200], loss:-1.4047
epoch [33/200], loss:-1.3485
epoc

In [16]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

tank_info_panel = 0
drone_info_panel = 23

def val_pos(array):
    value = np.sum(array)
    position = np.unravel_index(np.argmax(array),array.shape)
    return value,position

#Blank
img = uframes[0]
z = Image.fromarray(img)
z.show()
data3=[]
data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
y3 = np.array(data3)
x_1 = torch.from_numpy(y3)
output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
output_tank = output[0][tank_info_panel]
val,pos = val_pos(output_tank)
print("Blank: ",val,pos)
output_drone = output[0][drone_info_panel]
val,pos = val_pos(output_drone)
print("Blank: ",val,pos)


#Tank

img = Image.open("frame/tank_frame0.jpg")
img =  cv2.resize(np.asarray(img),(100,100))
data3=[]
data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
y3 = np.array(data3)
x_1 = torch.from_numpy(y3)
output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
output_tank = output[0][tank_info_panel]
val,pos = val_pos(output_tank)
print("Tank: ",val,pos)
output_drone = output[0][drone_info_panel]
val,pos = val_pos(output_drone)
print("Tank: ",val,pos)


#Drone

img = Image.open("frame/frame0.jpg")
img =  cv2.resize(np.asarray(img),(100,100))
data3=[]
data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
y3 = np.array(data3)
x_1 = torch.from_numpy(y3)
output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
output_tank = output[0][tank_info_panel]
val,pos = val_pos(output_tank)
print("Drone: ",val,pos)
output_drone = output[0][drone_info_panel]
val,pos = val_pos(output_drone)
print("Drone: ",val,pos)


#Tank and Drone
img = uframes[10]
z = Image.fromarray(img)
z.show()
data3=[]
data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
y3 = np.array(data3)
x_1 = torch.from_numpy(y3)
output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
output_tank = output[0][tank_info_panel]
val,pos = val_pos(output_tank)
print("TD: ",val,pos)
output_drone = output[0][drone_info_panel]
val,pos = val_pos(output_drone)
print("TD: ",val,pos)



Blank:  3263.7158 (2, 0)
Blank:  3012.2852 (10, 2)
Tank:  904.21124 (2, 2)
Tank:  256.77917 (1, 1)
Drone:  476.2919 (11, 2)
Drone:  128.64597 (11, 12)
TD:  2604.5034 (2, 7)
TD:  2405.1414 (1, 13)


In [46]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

drone_panels={}
tank_panels={}
tank_reloading_panels={}
drone_reloading_panels={}

def val_pos(array):
    value = np.sum(array)
    position = np.unravel_index(np.argmax(array),array.shape)
    return value,position

for i in range(32):

    tank_info_panel = i
    #rone_info_panel = i
    #Blank
    img = uframes[1]
    z = Image.fromarray(img)
    #z.show()
    data3=[]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
    output_tank = output[0][tank_info_panel]
    val,pos = val_pos(output_tank)
    print("Blank: ",val,pos)
#     output_drone = output[0][drone_info_panel]
#     val,pos = val_pos(output_drone)
#     print("Blank: ",val,pos)


    print("**********")
    img = uframes[2]
    z = Image.fromarray(img)
    #z.show()
    data3=[]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
    output_tank = output[0][tank_info_panel]
    val_tank_info,pos_tank_info = val_pos(output_tank)
    print("Tank: ",val_tank_info,pos_tank_info)
#     output_drone = output[0][drone_info_panel]
#     val_tank_drone_info,pos_tank_drone_info = val_pos(output_drone)
#     print("Tank: ",val,pos)
    print("**********")

    #Drone
    img = uframes[3]
    z = Image.fromarray(img)
    #z.show()
    data3=[]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
    output_tank = output[0][tank_info_panel]
    val_drone_info,pos_drone_info = val_pos(output_tank)
    print("Drone: ",val_drone_info,pos_drone_info)
#     output_drone = output[0][drone_info_panel]
#     val_drone_drone_info,pos_drone_drone_info = val_pos(output_drone)
#     print("Drone: ",val,pos)
    
    print("**********")  
    img = uframes[4]
    z = Image.fromarray(img)
    #z.show()
    data3=[]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
    output_tank = output[0][tank_info_panel]
    val_tank_reloading_info,pos_tank_reloading_info = val_pos(output_tank)
    print("Tank reloading: ",val_tank_reloading_info,pos_tank_reloading_info)
#     output_drone = output[0][drone_info_panel]
#     val_tank_reloading_drone_info,pos_tank_reloading_drone_info = val_pos(output_drone)
#     print("Tank reloading: ",val,pos)

    print("**********")
    #Drone Reloadingg
    img = uframes[5]
    z = Image.fromarray(img)
    #z.show()
    data3=[]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
    output_tank = output[0][tank_info_panel]
    val_drone_reloading_info,pos_drone_reloading_info = val_pos(output_tank)
    print("Drone reloading: ",val_drone_reloading_info,pos_drone_reloading_info)
#     output_drone = output[0][drone_info_panel]
#     val_drone_reloading_drone_info,pos_drone_reloading_drone_info = val_pos(output_drone)
#     print("Drone reloading: ",val,pos)

    print("**********")
    #Tank and Drone
    img= uframes[0]
    z = Image.fromarray(img)
    #z.show()
    #z.show()
    data3=[]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
    output_tank = output[0][tank_info_panel]
    val,pos = val_pos(output_tank)
    print("TD: ",val,pos)
#     output_drone = output[0][drone_info_panel]
#     val,pos = val_pos(output_drone)
#     print("TD: ",val,pos)
    
    print("**********")
    #Tank and Drone reloading
    img= uframes[6]
    z = Image.fromarray(img)
    #z.show()
    #z.show()
    data3=[]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    output = model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()
    output_tank = output[0][tank_info_panel]
    val,pos = val_pos(output_tank)
    print("TrDr: ",val,pos)
#     output_drone = output[0][drone_info_panel]
#     val,pos = val_pos(output_drone)
#     print("TD: ",val,pos)
    
    print("\nPanel ",i)
    print("\n++++++++++++++++++++++++++++++++++++++++++++\n")
    if val_tank_info > max(val_drone_reloading_info,val_tank_reloading_info,val_drone_info):
        tank_panels[i]=val_tank_info-5
    elif val_drone_info > max(val_drone_reloading_info,val_tank_reloading_info,val_tank_info):
        drone_panels[i]=val_drone_info-5
    elif val_tank_reloading_info > max(val_drone_reloading_info,val_tank_info,val_drone_info):
        tank_reloading_panels[i]=val_tank_reloading_info-5
    elif val_drone_reloading_info > max(val_tank_reloading_info,val_tank_info,val_drone_info):
        drone_reloading_panels[i]=val_drone_reloading_info-5
print(t_panels)

Blank:  1941.8152 (2, 0)
**********
Tank:  2214.7734 (6, 0)
**********
Drone:  2142.3945 (2, 0)
**********
Tank reloading:  2442.081 (5, 0)
**********
Drone reloading:  2367.435 (12, 10)
**********
TD:  2409.738 (6, 0)
**********
TrDr:  2868.833 (5, 0)

Panel  0

++++++++++++++++++++++++++++++++++++++++++++

Blank:  2050.3271 (9, 9)
**********
Tank:  2346.3916 (9, 9)
**********
Drone:  2335.3293 (9, 9)
**********
Tank reloading:  2546.259 (5, 0)
**********
Drone reloading:  2313.1282 (9, 9)
**********
TD:  2636.8242 (9, 9)
**********
TrDr:  2812.8105 (5, 0)

Panel  1

++++++++++++++++++++++++++++++++++++++++++++

Blank:  48.779747 (8, 8)
**********
Tank:  101.972145 (5, 1)
**********
Drone:  88.87184 (11, 10)
**********
Tank reloading:  75.83423 (6, 1)
**********
Drone reloading:  71.340675 (10, 10)
**********
TD:  140.83664 (5, 1)
**********
TrDr:  98.10787 (10, 10)

Panel  2

++++++++++++++++++++++++++++++++++++++++++++

Blank:  951.79944 (12, 3)
**********
Tank:  1180.1526 (6, 0)
**

Tank reloading:  2063.8513 (5, 0)
**********
Drone reloading:  2196.0337 (11, 11)
**********
TD:  2438.5913 (11, 10)
**********
TrDr:  2545.3499 (11, 11)

Panel  26

++++++++++++++++++++++++++++++++++++++++++++

Blank:  2089.476 (12, 13)
**********
Tank:  2442.4312 (5, 0)
**********
Drone:  2492.7466 (10, 10)
**********
Tank reloading:  2371.3477 (12, 13)
**********
Drone reloading:  2576.0852 (11, 10)
**********
TD:  2845.8218 (5, 0)
**********
TrDr:  2855.54 (11, 10)

Panel  27

++++++++++++++++++++++++++++++++++++++++++++

Blank:  1109.6542 (12, 3)
**********
Tank:  1340.8745 (5, 0)
**********
Drone:  1413.801 (11, 10)
**********
Tank reloading:  1563.3765 (6, 2)
**********
Drone reloading:  1272.1742 (12, 3)
**********
TD:  1629.2128 (11, 10)
**********
TrDr:  1723.7188 (6, 2)

Panel  28

++++++++++++++++++++++++++++++++++++++++++++

Blank:  2057.3538 (16, 15)
**********
Tank:  2360.1306 (16, 15)
**********
Drone:  2377.775 (11, 9)
**********
Tank reloading:  2380.6377 (16, 15)
***

In [55]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

for j in range(len(list(drone_reloading_panels))):
    
    tank_info_panel = list(tank_panels)[0]
    drone_info_panel = list(drone_panels)[0]
    tankr_info_panel = list(tank_reloading_panels)[j]
    droner_info_panel = list(drone_reloading_panels)[0] #12

    #print(tankr_info_panel,tank_reloading_panels[tankr_info_panel])

    tank_threshold= tank_panels[tank_info_panel]
    tankr_threshold= tank_reloading_panels[tankr_info_panel]
    drone_threshold= drone_panels[drone_info_panel]
    droner_threshold= drone_reloading_panels[droner_info_panel]

    def val_pos(array):
        value = np.sum(array)
        position = np.unravel_index(np.argmax(array),array.shape)
        return value,position
    dronec=[]
    tankc=[]
    tankr=[]
    droner=[]
    #Blank
    for i in range(50):
        #x = cv2.imread(r"C:\Users\darshan19\Desktop\AoI\CNN\Images1\0b.png",-1)
        #x = cv2.resize(x,(100,100))
        img = uframes[i]
        #print(np.shape(img))
        # = Image.fromarray(img)
        #z.show()
        data3=[]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        #print(np.shape(x_1))
        output = model(x_1.reshape(1,3,100, 100))[0].detach().numpy()
        output_tank = output[0][tank_info_panel]
        val_tank,pos_tank = val_pos(output_tank)
        output_tankr = output[0][tankr_info_panel]
        val_tankr,pos_tankr = val_pos(output_tankr)
        print(i)
        output_drone = output[0][drone_info_panel]
        val_drone,pos_drone = val_pos(output_drone)
        output_droner = output[0][droner_info_panel]
        val_droner,pos_droner = val_pos(output_droner)

        if val_tank>tank_threshold:
    #         print("tank present")
    #         print("Tank info: ",val_tank,pos_tank)
            tankc.append(pos_tank)
        if val_drone>drone_threshold:
    #         print("drone present")
    #         print("Drone info: ",val_drone,pos_drone)
            dronec.append(pos_drone)
        if val_tankr>tankr_threshold:
            print("tank reloading present")
            print("Tank info: ",val_tank,pos_tank)
            tankr.append(pos_tankr)
        if val_droner>droner_threshold:
#             print("drone reloading present")
#             print("Drone info: ",val_droner,pos_droner)
            dronec.append(pos_drone)
    print("\n")
    print(list(tank_reloading_panels)[j])
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n")


IndexError: list index out of range

In [54]:
img = uframes[6]
z = Image.fromarray(img)
z.show()

In [15]:
im1 = Image.open('tree1.png')
im1 = im1.resize((100,100))

imt = Image.open('tank.png')
imt = imt.resize((100,100))


im2 = Image.open('tree2.png')
im2 = im2.resize((100,100))

im3 = Image.open('tree3.png')
im3 = im3.resize((100,100))

im_drone = Image.open('drone4-1.png')
im_drone = im_drone.resize((200,200))

imb = Image.open('bullet.png')
imb = imb.resize((40,40))



img_blank = Image.new('RGB', (720, 1080), color = '#9b7653')
img_blank.paste(im1,(5,75),im1)
img_blank.paste(im1,(450,550),im1)
img_blank.paste(im1,(205,720),im1)
img_blank.paste(im2,(500,900),im2)
img_blank.paste(im2,(650,750),im2)
img_blank.paste(im2,(320,440),im2)
img_blank.paste(im3,(100,600),im3)
img_blank.paste(im3,(440,620),im3)
img_blank.paste(im3,(300,150),im3)
img_blank.paste(imt.rotate(180),(500,500),imt.rotate(180))
img_blank = np.asarray(img_blank)
img_blank = cv2.resize(img_blank,(100,100))
pil = np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32)
pil = torch.from_numpy(pil)
output = model(pil.reshape(1,3,100, 100))[0].detach().numpy()
output = output[0][info_panel]
val,pos = val_pos(output)
print(val,pos)

NameError: name 'info_panel' is not defined

In [9]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

for i in range(1):
    img = uframes[5]
    #print(np.shape(img))
    z = Image.fromarray(img)
    z.show()

In [4]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

filename = "Marginalization_Simple-increased_channels-32/fc_weights.txt"
file = open(filename, 'w')
file.write(str(model.fc1.weight.detach().numpy()))
print(np.shape(model.fc1.weight.detach().numpy()))
file.close()


(15, 10368)


In [4]:
##Weights

import operator
k=model.fc1.weight.detach().numpy()
for i in range(15):
    index, value = max(enumerate(k[i]), key=operator.itemgetter(1))
    print("FC - Feature "+ "i: ",index,value)
    print("CNN value: ", )

FC - Feature i:  684 0.041578084
CNN value: 
FC - Feature i:  684 0.051804464
CNN value: 
FC - Feature i:  360 0.04816093
CNN value: 
FC - Feature i:  492 0.035558272
CNN value: 
FC - Feature i:  478 0.06459166
CNN value: 
FC - Feature i:  585 0.049136233
CNN value: 
FC - Feature i:  492 0.081621096
CNN value: 
FC - Feature i:  360 0.065480515
CNN value: 
FC - Feature i:  585 0.04792851
CNN value: 
FC - Feature i:  492 0.06556648
CNN value: 
FC - Feature i:  684 0.035789683
CNN value: 
FC - Feature i:  684 0.034514345
CNN value: 
FC - Feature i:  817 0.043642323
CNN value: 
FC - Feature i:  492 0.088237286
CNN value: 
FC - Feature i:  492 0.09752004
CNN value: 


In [6]:
import operator
import sys
#t1d1
img_blank = Image.new('RGB', (100, 100), color = '#9b7653')
img_blank.paste(imt.rotate(180),(10,30),imt.rotate(180))
# img_blank.paste(im_drone.resize((10,10)),(2,10),im_drone.resize((10,10)))

# #t2d2
# img_blank.paste(imt.rotate(180),(30,30),imt.rotate(180))
# draw = ImageDraw.Draw(img_blank)
# draw.ellipse((35,35,38,38), fill=(255, 0, 0))
# img_blank.paste(im_drone.resize((10,10)),(35,45),im_drone.resize((10,10)))
# draw.ellipse((40,50,43,53), fill=(50, 182, 150))

# #t3d3
# img_blank.paste(imt.rotate(180),(50,30),imt.rotate(180))
# img_blank.paste(imb.resize((2,2)),(54,40),imb.resize((2,2)))
# img_blank.paste(im_drone.resize((10,10)),(50,70),im_drone.resize((10,10)))
# img_blank.paste(imb.resize((2,2)),(56,81),imb.resize((2,2)))

v = np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32)
v = torch.from_numpy(v)
k = model(v.reshape(1,3,100, 100))[0].view(model(v.reshape(1,3,100, 100))[0].size(0), -1).detach().numpy()

filename = "Marginalization_Simple/flattened_tank_pos_1.txt"
file = open(filename, 'w')
file.write(str(k))
file.close()


# print(np.shape(k))
# index, value = max(enumerate(k), key=operator.itemgetter(1))
# print(index,value)


import operator
p=model.fc1.weight.detach().numpy()
for i in range(15):
    index, value = max(enumerate(p[i]), key=operator.itemgetter(1))
    print("FC - Feature "+ str(i) +" : ",index,value)
    print
    print("CNN value: ",k[0][index] )

FC - Feature 0 :  684 0.041578084
CNN value:  17.59211
FC - Feature 1 :  684 0.051804464
CNN value:  17.59211
FC - Feature 2 :  360 0.04816093
CNN value:  0.53643996
FC - Feature 3 :  492 0.035558272
CNN value:  0.53643996
FC - Feature 4 :  478 0.06459166
CNN value:  0.53643996
FC - Feature 5 :  585 0.049136233
CNN value:  0.53643996
FC - Feature 6 :  492 0.081621096
CNN value:  0.53643996
FC - Feature 7 :  360 0.065480515
CNN value:  0.53643996
FC - Feature 8 :  585 0.04792851
CNN value:  0.53643996
FC - Feature 9 :  492 0.06556648
CNN value:  0.53643996
FC - Feature 10 :  684 0.035789683
CNN value:  17.59211
FC - Feature 11 :  684 0.034514345
CNN value:  17.59211
FC - Feature 12 :  817 0.043642323
CNN value:  17.59211
FC - Feature 13 :  492 0.088237286
CNN value:  0.53643996
FC - Feature 14 :  492 0.09752004
CNN value:  0.53643996


In [21]:

import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))[1]
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
length  600


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init  
import time
import cv2

print(np.shape(uframes[0]))


#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
print("Length of Class data: ", len(class_data))
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

print("Start")

img_blank = Image.new('RGB', (100, 100), color = '#9b7653')
img_blank.paste(im1,(5,7),im1)
img_blank.paste(im1,(45,55),im1)
img_blank.paste(im1,(25,72),im1)
img_blank.paste(im2,(50,90),im2)
img_blank.paste(im2,(60,75),im2)
img_blank.paste(im2,(30,40),im2)
img_blank.paste(im3,(10,60),im3)
img_blank.paste(im3,(44,62),im3)
img_blank.paste(im3,(30,15),im3)
img_blank.paste(imt.rotate(180),(50,30),imt.rotate(180))
img_blank.paste(imb.resize((2,2)),(54,40),imb.resize((2,2)))
#img_blank.paste(im_drone.resize((10,10)),(50,70),im_drone.resize((10,10)))
#img_blank.paste(imb.resize((2,2)),(56,81),imb.resize((2,2)))

for i in range(12):
    #z = cv2.imread('Images_100/0a.png')
    #l = Image.fromarray(z)
#     print("shape of z", np.shape(z),type(z))
#     #l.show()
#     #z  = cv2.resize(z,(100,100))
#     img = uframes[i]
#     print(np.shape(img))
#     img.show()
#     data3.append(np.array(np.transpose(np.asarray(uframes[i]),(2,0,1)),dtype=np.float32))
#     y3 = np.array(data3)
#     x_1 = torch.from_numpy(y3)
# #     random_img = random.choice(x)
# #     print("random", np.shape(random_img),type(random_img))
# #     show = random_img.detach().numpy() * 255
# #     show = np.array(np.transpose(show,(1,2,0)),dtype=np.uint8)
# #     print("show", np.shape(show),type(show))
# #     ms = Image.fromarray(show)
# #     ms.show()
#     temp = model(x_1[i].reshape(1,3,100,100))[1]
#     temp = temp.detach().numpy()
#     img_blank.show()
#     #print(temp)
#     z = clf.predict(temp)
#     print(names[z[0]])
    img = uframes[i]
    print(np.shape(img))
    img.show()
    #ms = Image.fromarray(z)
    #ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))[1]
    temp = temp.detach().numpy()
    print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
    
    

(100, 100, 3)
Length of Class data:  600
Start
(100, 100, 3)
[[5.69654167e-01 9.99462187e-01 5.63863292e-02 4.24214751e-01
  9.99946237e-01 3.76719624e-01 9.24754981e-03 9.96866763e-01
  3.24720666e-02 6.23485744e-01 9.99366581e-01 1.33173708e-05
  9.99919176e-01 1.10880566e-04 7.16020440e-05]]
Just Background
(100, 100, 3)
[[1.0693459e-01 9.7853309e-01 6.2482929e-01 8.8721722e-01 9.2732191e-01
  3.8686104e-04 4.0334295e-03 9.7163016e-01 4.2039016e-01 9.9970645e-01
  9.6105587e-01 4.5536429e-02 9.1955161e-01 2.7226047e-03 7.8407951e-02]]
Only Tank moving
(100, 100, 3)
[[7.3534204e-03 9.9773276e-01 9.8508829e-01 9.9262118e-01 4.5104271e-01
  3.5262175e-04 1.2917872e-04 7.4658793e-01 9.1725689e-01 9.9936992e-01
  9.9860114e-01 5.1300049e-01 5.2829486e-01 2.9081654e-02 4.6734187e-01]]
Only Drone Moving
(100, 100, 3)
[[6.6964526e-04 9.1521192e-01 9.9945718e-01 9.9930429e-01 5.6288450e-04
  2.2586556e-07 5.6052031e-05 2.4077968e-01 9.9584311e-01 9.9999964e-01
  9.1780317e-01 9.9973506e-01 1

In [66]:
print(len(uframes
         ))

12


In [44]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

img_blank = Image.new('RGB', (100, 100), color = '#9b7653')
img_blank.paste(im1,(5,7),im1)
img_blank.paste(im1,(45,55),im1)
img_blank.paste(im1,(25,72),im1)
img_blank.paste(im2,(50,90),im2)
img_blank.paste(im2,(60,75),im2)
img_blank.paste(im2,(30,40),im2)
img_blank.paste(im3,(10,60),im3)
img_blank.paste(im3,(44,62),im3)
img_blank.paste(im3,(30,15),im3)
img_blank.paste(imt.rotate(180),(60,60),imt.rotate(180))
#img_blank.paste(im_drone.resize((10,10)),(10,30),im_drone.resize((10,10)))
#img_blank.paste(imt.rotate(180),(30,30),imt.rotate(180))
#draw = ImageDraw.Draw(img_blank)
#draw.ellipse((60,60,68,68), fill=(255, 0, 0))
# img_blank.paste(im_drone.resize((10,10)),(35,45),im_drone.resize((10,10)))
#draw = ImageDraw.Draw(img_blank)
#draw.ellipse((10,30,18,38), fill=(50, 182, 150))
#img_blank.paste(imt.rotate(180),(50,30),imt.rotate(180))
#img_blank.paste(imb.resize((2,2)),(54,40),imb.resize((2,2)))
#img_blank.paste(im_drone.resize((10,10)),(50,70),im_drone.resize((10,10)))
#img_blank.paste(imb.resize((2,2)),(56,81),imb.resize((2,2)))
img_blank.show()
pil = np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32)
pil = torch.from_numpy(pil)
filename = "lesser_15_features/testing.txt"
file = open(filename, 'w')
file.write(str(model(pil.reshape(1,3,100, 100))[0].detach().numpy()))
file.close()

In [9]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
img_blank = Image.new('RGB', (100, 100), color = '#9b7653')
#img_blank.paste(imt.rotate(180),(30,30),imt.rotate(180))
#img_blank.paste(im_drone.resize((10,10)),(35,45),im_drone.resize((10,10)))

#img_blank.paste(imt.rotate(180),(50,30),imt.rotate(180))
img_blank.paste(im_drone.resize((10,10)),(50,70),im_drone.resize((10,10)))

pil = np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32)
pil = torch.from_numpy(pil)
filename = "Marginalization_Simple-increased_channels-32/drone_pos_3.txt"
file = open(filename, 'w')
file.write(str(model(pil.reshape(1,3,100, 100))[0].detach().numpy()))
file.close()

In [4]:
import cv2
vidcap = cv2.VideoCapture('tank.avi')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame/tank_frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [6]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)   

from PIL import  Image,ImageDraw

img = uframes[20]
z = Image.fromarray(img)
z.show()
data3=[]
data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
y3 = np.array(data3)
x_1 = torch.from_numpy(y3)
temp = model(x_1[0].reshape(1,3,100,100))[0].detach().numpy()
filename = "Marginalization_Simple-increased_channels-32-video-multiple/TD.txt"
file = open(filename, 'w')
file.write(str(model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()))
file.close()

In [14]:
import sys
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)   

from PIL import  Image,ImageDraw

img = Image.open("frame/tank_frame0.jpg")
img =  cv2.resize(np.asarray(img),(100,100))
data3=[]
data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
y3 = np.array(data3)
x_1 = torch.from_numpy(y3)
temp = model(x_1[0].reshape(1,3,100,100))[0].detach().numpy()
filename = "Marginalization_Simple-increased_channels-32-video-multiple/tank.txt"
file = open(filename, 'w')
file.write(str(model(x_1[0].reshape(1,3,100, 100))[0].detach().numpy()))
file.close()